Write a Python program that simulates a music streaming service. Your system should support the following functionalities:

Users can "subscribe" to different music genres. Users can view their subscribed genres and discover new music. Users can play a list of songs from their subscribed genres. To implement this system, you will need to gather songs from various genres that are available for free or in the public domain on the Internet. Each genre should have a collection of songs that users can browse and play.

Requirements: Implement a MusicStreamingService class with methods for subscribing to genres, viewing subscriptions, and playing songs. Create a Song class that includes attributes like title, artist, and duration. Ensure that users can browse available genres and the songs within each genre before making a subscription. Use an external source (like a public domain music database or website) to populate your list of available songs for users to explore. Bonus: Allow users to create custom playlists based on their subscribed genres and save them for later playback. Implement a feature to search for songs by title or artist.

In [3]:
import requests

class Song:
    def __init__(self, title, artist, duration):
        self.title = title
        self.artist = artist
        self.duration = duration

    def __str__(self):
        return f"{self.title} by {self.artist} ({self.duration} min)"

class MusicStreamingService:
    def __init__(self, api_base_url, api_key):
        self.api_base_url = api_base_url
        self.api_key = api_key
        self.subscribed_genres = set() 
        self.genre_songs = {}           

    def _make_api_request(self, endpoint, params=None):
        """Helper method to make an API request."""
        url = f"https://api.discogs.com/releases/249504"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Accept": "FooBarApp/3.0"
        } 
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data from API. Status code: {response.status_code}")
            return None

    def fetch_available_genres(self):
        """Fetch available genres from the API."""
        data = self._make_api_request("genres")
        if data:
            genres = data.get("genres", [])
            print("Available Genres:")
            for genre in genres:
                print("-", genre)
            return genres
        else:
            return []

    def subscribe_to_genre(self, genre):
        """Subscribe the user to a specific genre."""
        available_genres = self.fetch_available_genres()
        if genre in available_genres:
            self.subscribed_genres.add(genre)
            print(f"Subscribed to {genre}")
        else:
            print(f"Genre '{genre}' is not available.")

    def view_subscriptions(self):
        """View the user's subscribed genres."""
        if self.subscribed_genres:
            print("Subscribed Genres:")
            for genre in self.subscribed_genres:
                print("-", genre)
        else:
            print("No subscriptions yet.")

    def fetch_songs_by_genre(self, genre):
        """Fetch songs for a specific genre."""
        if genre in self.genre_songs:
            return self.genre_songs[genre]

        data = self._make_api_request("songs", params={"genre": genre})
        if data:
            songs = data.get("songs", [])
            song_list = [Song(song["title"], song["artist"], song["duration"]) for song in songs]
            self.genre_songs[genre] = song_list  # Cache the songs
            return song_list
        else:
            return []

    def play_songs(self, genre):
        """Play songs from the subscribed genres."""
        if genre not in self.subscribed_genres:
            print(f"You are not subscribed to the genre '{genre}'.")
            return

        songs = self.fetch_songs_by_genre(genre)
        if songs:
            print(f"Playing songs from {genre}:")
            for song in songs:
                print(song)
        else:
            print(f"No songs found for genre '{genre}'.")

def cli():
    api_base_url = "https://api.example.com/v1/music"
    api_key = "YOUR_API_KEY"

    service = MusicStreamingService(api_base_url, api_key)

    print("Welcome to the Music Streaming Service!")
    print("Available commands: genres, subscribe, subscriptions, play, exit")

    while True:
        command = input("\nEnter a command: ").strip().lower()

        if command == "genres":
            service.fetch_available_genres()

        elif command == "subscribe":
            genre_to_subscribe = input("Enter a genre to subscribe: ").strip()
            service.subscribe_to_genre(genre_to_subscribe)

        elif command == "subscriptions":
            service.view_subscriptions()

        elif command == "play":
            genre_to_play = input("Enter a subscribed genre to play songs: ").strip()
            service.play_songs(genre_to_play)

        elif command == "exit":
            print("Exiting the Music Streaming Service. Goodbye!")
            break

        else:
            print("Invalid command. Please try again.")

if __name__ == "__main__":
    cli()

Welcome to the Music Streaming Service!
Available commands: genres, subscribe, subscriptions, play, exit

Enter a command: genres
Available Genres:
- Electronic
- Pop

Enter a command: subscribe
Enter a genre to subscribe: Electronic
Available Genres:
- Electronic
- Pop
Subscribed to Electronic

Enter a command: play
Enter a subscribed genre to play songs: Electronic
No songs found for genre 'Electronic'.

Enter a command: subscribe
Enter a genre to subscribe: Pop
Available Genres:
- Electronic
- Pop
Subscribed to Pop

Enter a command: play
Enter a subscribed genre to play songs: pop
You are not subscribed to the genre 'pop'.

Enter a command: play
Enter a subscribed genre to play songs: Pop
No songs found for genre 'Pop'.

Enter a command: exit
Exiting the Music Streaming Service. Goodbye!
